In [1]:
import mne
import os
import numpy as np
import warnings
from mne.preprocessing import ICA
import mne_icalabel
import matplotlib.pyplot as plt
from mne.channels import make_dig_montage
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.mean import mean_riemann
from mne.decoding import CSP
from scipy.signal import welch
import pandas as pd
import gc
import warnings
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, confusion_matrix,
                            ConfusionMatrixDisplay, RocCurveDisplay,
                            PrecisionRecallDisplay, log_loss)

# Suppress warnings and set MNE logging to minimal
warnings.filterwarnings('ignore')
mne.set_log_level('WARNING')

In [ ]:
# def create_eegmmidb_montage():
#     """
#     Create a custom montage for the 14 selected channels based on standard_1020.
#     """
#     emotiv_channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']

#     standard_montage = mne.channels.make_standard_montage('standard_1020')

#     ch_pos = {}0
#     for ch in emotiv_channels:
#         try:
#             pos = standard_montage.dig[standard_montage.ch_names.index(ch)]['r']
#             ch_pos[ch] = pos
#         except ValueError:
#             print(f"Warning: Position for channel {ch} not found in standard_1020. Skipping.")

#     custom_montage = make_dig_montage(ch_pos=ch_pos, coord_frame='head')
#     return custom_montage

# def preprocess_data(subject, subject_folder, output_folder, epoch_counts):
#     """
#     Preprocess EEG data from EEGMMIDB for left_fist (T1, runs 4,8,12) and both_feet (T2, runs 6,10,14).
#     Track epoch counts for left_fist and both_feet tasks.
#     """
#     all_epochs_list = []

#     # Define runs
#     left_fist_runs = [4, 8, 12]  # T1: left_fist imagined
#     both_feet_runs = [6, 10, 14]  # T2: both_feet imagined
#     all_runs = left_fist_runs + both_feet_runs

#     for run in all_runs:
#         edf_file = os.path.join(subject_folder, f'{subject}R{run:02d}.edf')
#         if not os.path.exists(edf_file):
#             print(f"File {edf_file} not found, skipping.")
#             continue

#         try:
#             # 1. Data Loading and Channel Setup
#             raw = mne.io.read_raw_edf(edf_file, preload=True, verbose=False)
#             print(f"Raw channel names before renaming in {edf_file}: {raw.ch_names}")
#             raw.rename_channels({ch: ch.rstrip('.').upper() for ch in raw.ch_names})
#             print(f"Raw channel names after renaming in {edf_file}: {raw.ch_names}")

#             # Select only the specified 14 channels
#             emotiv_channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1',
#                                'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
#             raw.pick_channels(emotiv_channels, ordered=True)

#             raw.set_eeg_reference('average', verbose=False)
#             montage = create_eegmmidb_montage()
#             raw.set_montage(montage, on_missing='ignore')
#             raw.filter(4, 40, method='fir', phase='zero-double', verbose=False)

#             # 2. ICA Processing
#             n_components = min(len(raw.ch_names), 14)  # Adjusted for 14 channels
#             ica = ICA(n_components=n_components, method='picard', max_iter=1000, random_state=42)
#             ica.fit(raw, reject_by_annotation=True, verbose=False)
#             ic_labels = mne_icalabel.label_components(raw, ica, method='iclabel')
#             ica.exclude = [i for i, label in enumerate(ic_labels['labels'])
#                           if label in ('eye blink', 'muscle artifact', 'heart beat', 'line noise')]
#             raw = ica.apply(raw, verbose=False)

#             # 3. Event Extraction and Epoch Creation
#             events = []
#             annotations = raw.annotations.description
#             unique_annotations = set(annotations)
#             print(f"Annotations in {edf_file}: {unique_annotations}")

#             # Count relevant events only
#             if run in left_fist_runs:
#                 t_count = sum(1 for ann in annotations if ann == 'T1')
#                 print(f"Event counts in {edf_file}: T1 (left_fist) = {t_count}")
#                 event_id = {'left_fist': 0}
#             else:  # both_feet_runs
#                 t_count = sum(1 for ann in annotations if ann == 'T2')
#                 print(f"Event counts in {edf_file}: T2 (both_feet) = {t_count}")
#                 event_id = {'both_feet': 1}

#             for ann in raw.annotations:
#                 desc = ann['description']
#                 event_code = None
#                 # Only process T1 for left_fist_runs, T2 for both_feet_runs
#                 if run in left_fist_runs and desc == 'T1':
#                     event_code = 0  # left_fist
#                 elif run in both_feet_runs and desc == 'T2':
#                     event_code = 1  # both_feet

#                 if event_code is not None:
#                     onset_sample = int(ann['onset'] * raw.info['sfreq'])
#                     events.append([onset_sample, 0, event_code])

#             if len(events) < 4:
#                 print(f"Skipping {edf_file} - insufficient events ({len(events)})")
#                 continue

#             epochs = mne.Epochs(
#                 raw,
#                 np.array(events),
#                 event_id=event_id,
#                 tmin=-0.5,
#                 tmax=3.5,
#                 baseline=(-0.5, 0),
#                 preload=True,
#                 reject=dict(eeg=200e-6),
#                 verbose=False
#             )

#             epochs = epochs.crop(tmin=0, tmax=3).resample(128)
#             epochs.baseline = None
#             all_epochs_list.append(epochs)

#         except Exception as e:
#             print(f"Error processing {edf_file}: {str(e)}")
#             continue

#     if all_epochs_list:
#         try:
#             # Ensure consistent event_id when combining epochs
#             for epochs in all_epochs_list:
#                 if epochs.event_id == {'left_fist': 0}:
#                     epochs.event_id = {'left_fist': 0, 'both_feet': 1}
#                     epochs.events[epochs.events[:, 2] == 0, 2] = 0
#                 elif epochs.event_id == {'both_feet': 1}:
#                     epochs.event_id = {'left_fist': 0, 'both_feet': 1}
#                     epochs.events[epochs.events[:, 2] == 1, 2] = 1

#             combined_epochs = mne.concatenate_epochs(all_epochs_list)
#             epochs_fname = os.path.join(output_folder, f'{subject}_eegmmidb_epochs-epo.fif')
#             combined_epochs.save(epochs_fname, overwrite=True, verbose=False)

#             left_count = sum(combined_epochs.events[:, 2] == 0)
#             feet_count = sum(combined_epochs.events[:, 2] == 1)
#             epoch_counts[subject] = {'left_fist': left_count, 'both_feet': feet_count}

#             print(f"\nProcessed {len(combined_epochs)} epochs with {len(combined_epochs.info['ch_names'])} channels:")
#             print(combined_epochs.info['ch_names'])
#             print(f"Class balance - Left Fist: {left_count}, Both Feet: {feet_count}")
#         except Exception as e:
#             print(f"Error combining epochs for {subject}: {str(e)}")

# def plot_epoch_counts(epoch_counts):
#     """
#     Plot the number of epochs for each task per subject as a grouped bar plot.
#     """
#     if not epoch_counts:
#         print("No epoch counts to plot.")
#         return

#     subjects = list(epoch_counts.keys())
#     fists_counts = [epoch_counts[subj]['left_fist'] for subj in subjects]
#     feet_counts = [epoch_counts[subj]['both_feet'] for subj in subjects]

#     bar_width = 0.35
#     index = np.arange(len(subjects))

#     plt.figure(figsize=(12, 6))
#     plt.bar(index, fists_counts, bar_width, label='Left Fist', color='skyblue')
#     plt.bar(index + bar_width, feet_counts, bar_width, label='Both Feet', color='lightcoral')

#     plt.xlabel('Subject')
#     plt.ylabel('Number of Epochs')
#     plt.title('Number of Epochs per Task for Each Subject')
#     plt.xticks(index + bar_width / 2, subjects, rotation=45)
#     plt.legend()
#     plt.tight_layout()
#     plt.savefig('epoch_counts.png')
#     plt.close()
#     print("Epoch counts plot saved to 'epoch_counts.png'")

# def main():
#     """Main processing routine"""
#     data_dir = 'files'
#     output_folder = 'processed_eegmmidb_data'
#     os.makedirs(output_folder, exist_ok=True)

#     epoch_counts = {}
#     subjects = [d for d in os.listdir(data_dir)
#                 if os.path.isdir(os.path.join(data_dir, d))]

#     for subject in subjects:
#         print(f"\n{'='*40}\nProcessing {subject}")
#         preprocess_data(subject, os.path.join(data_dir, subject), output_folder, epoch_counts)

#     print("\nProcessing completed. EEGMMIDB data ready for analysis.")
#     plot_epoch_counts(epoch_counts)

# if __name__ == '__main__':
#     main()

In [3]:
import mne
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from mne.decoding import CSP
import pandas as pd
import gc
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
mne.set_log_level('WARNING')

# Custom Transformer to Enforce float64
class EnsureFloat64(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_out = X.astype(np.float64)
        print(f"EnsureFloat64 output shape: {X_out.shape}, dtype: {X_out.dtype}")
        return X_out

# Tangent Space Features
class TangentSpaceFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, n_channels=14):
        self.n_channels = n_channels

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if X.dtype != np.float64:
            X = X.astype(np.float64)

        # Compute covariance matrices with Ledoit-Wolf estimator
        cov_estimator = Covariances(estimator='lwf')  # Use Ledoit-Wolf with built-in shrinkage
        cov_matrices = cov_estimator.fit_transform(X)

        # Optional: Add manual diagonal loading for extra stability
        reg = 1e-6  # Small regularization term
        for i in range(cov_matrices.shape[0]):
            cov_matrices[i] += np.eye(cov_matrices.shape[1]) * reg

        # Project to tangent space
        ts = TangentSpace(metric='riemann')
        tangent_features = ts.fit_transform(cov_matrices)

        print(f"Tangent space features shape: {tangent_features.shape}, dtype: {tangent_features.dtype}")
        gc.collect()
        return tangent_features

def extract_features():
    output_folder = 'processed_eegmmidb_data'
    feature_output = 'eeg_features.csv'
    sfreq = 128

    # Load epochs
    epoch_files = [f for f in os.listdir(output_folder) if f.endswith('-epo.fif')]
    all_epochs = []
    epoch_counts = []

    for file in epoch_files:
        file_path = os.path.join(output_folder, file)
        try:
            epochs = mne.read_epochs(file_path, preload=True, verbose=False)
            epochs = epochs.crop(tmin=0.0, tmax=3.0, include_tmax=False)
            epochs = epochs.resample(sfreq)
            if len(epochs) < 10:
                print(f"Skipping {file}: insufficient epochs ({len(epochs)})")
                continue
            print(f"Loaded {file}: {len(epochs)} epochs")
            all_epochs.append(epochs)
            epoch_counts.append(len(epochs))
        except Exception as e:
            print(f"Error loading {file_path}: {str(e)}")
            continue

    if not all_epochs:
        raise ValueError("No valid epochs loaded. Check preprocessing output.")

    combined_epochs = mne.concatenate_epochs(all_epochs)

    X = combined_epochs.get_data().astype(np.float64)
    y = combined_epochs.events[:, 2]  # Labels: 0 (left_fist), 1 (both_feet)

    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    if np.any(np.isinf(X)):
        raise ValueError("Data contains infinite values after preprocessing")

    print(f"Loaded {X.shape[0]} epochs, {X.shape[1]} channels, {X.shape[2]} time points")
    print(f"Class distribution - left_fist: {np.sum(y == 0)}, both_feet: {np.sum(y == 1)}")

    # Get channel names
    ch_names = combined_epochs.info['ch_names']
    expected_channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1',
                        'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
    if len(ch_names) != 14 or ch_names != expected_channels:
        raise ValueError(f"Expected 14 channels {expected_channels}, got {len(ch_names)}: {ch_names}")

    # Define pipelines
    csp_pipe = Pipeline([
        ('ensure_float64', EnsureFloat64()),
        ('csp', CSP(n_components=14, reg='ledoit_wolf', log=True)),  # Updated to 14 components
        ('csp_scaler', StandardScaler(copy=True)),
    ])

    tangent_pipe = Pipeline([
        ('tangent', TangentSpaceFeatures(n_channels=14)),
        ('tangent_scaler', StandardScaler(copy=True)),
    ])

    # Extract features
    print("Extracting CSP features...")
    csp_features = csp_pipe.fit_transform(X, y)
    print("Extracting tangent space features...")
    tangent_features = tangent_pipe.fit_transform(X, y)

    # Combine features
    features = np.hstack((csp_features, tangent_features))
    print(f"Combined features shape: {features.shape}")

    # Apply PCA
    pca = PCA(n_components=0.99, random_state=229)
    features = pca.fit_transform(features)
    print(f"PCA-reduced features shape: {features.shape}, explained variance: {sum(pca.explained_variance_ratio_):.4f}")

    # Create feature names
    n_csp = 14  # Updated to 14 components
    n_tangent = tangent_features.shape[1]
    feature_names = (
        [f'CSP_{i}' for i in range(n_csp)] +
        [f'Tangent_{i}' for i in range(n_tangent)]
    )

    pca_feature_names = [f'PC_{i}' for i in range(features.shape[1])]

    # Save to CSV
    feature_df = pd.DataFrame(features, columns=pca_feature_names)
    feature_df['label'] = y
    feature_df.to_csv(feature_output, index=False)
    print(f"Features saved to {feature_output}")

    # Clear memory
    del X, y, combined_epochs, csp_features, tangent_features, features, feature_df
    gc.collect()

if __name__ == '__main__':
    extract_features()

Loaded S001_eegmmidb_epochs-epo.fif: 47 epochs
Loaded S002_eegmmidb_epochs-epo.fif: 44 epochs
Loaded S003_eegmmidb_epochs-epo.fif: 47 epochs
Loaded S004_eegmmidb_epochs-epo.fif: 46 epochs
Loaded S005_eegmmidb_epochs-epo.fif: 43 epochs
Loaded S006_eegmmidb_epochs-epo.fif: 47 epochs
Loaded S007_eegmmidb_epochs-epo.fif: 46 epochs
Loaded S008_eegmmidb_epochs-epo.fif: 44 epochs
Loaded S009_eegmmidb_epochs-epo.fif: 46 epochs
Loaded S010_eegmmidb_epochs-epo.fif: 46 epochs
Loaded S011_eegmmidb_epochs-epo.fif: 46 epochs
Loaded S012_eegmmidb_epochs-epo.fif: 44 epochs
Loaded S013_eegmmidb_epochs-epo.fif: 46 epochs
Loaded S014_eegmmidb_epochs-epo.fif: 43 epochs
Loaded S015_eegmmidb_epochs-epo.fif: 46 epochs
Loaded S016_eegmmidb_epochs-epo.fif: 43 epochs
Loaded S017_eegmmidb_epochs-epo.fif: 44 epochs
Loaded S018_eegmmidb_epochs-epo.fif: 45 epochs
Loaded S019_eegmmidb_epochs-epo.fif: 46 epochs
Loaded S020_eegmmidb_epochs-epo.fif: 45 epochs
Loaded S021_eegmmidb_epochs-epo.fif: 45 epochs
Loaded S022_e

In [6]:
import joblib
# 1. Load Features
def load_features(feature_file):
    df = pd.read_csv(feature_file)
    X = df.drop('label', axis=1).values.astype(np.float64)
    y = df['label'].values.astype(np.int32)
    feature_names = df.drop('label', axis=1).columns.tolist()

    print(f"Loaded features shape: {X.shape}")
    print(f"Class distribution - Left Fist: {np.sum(y == 0)}, Both Feet: {np.sum(y == 1)}")
    return X, y, feature_names

# 2. Pipeline
def create_training_pipeline():
    return Pipeline([
        ('scaler', StandardScaler(copy=True)),
        ('pca', PCA(n_components=0.95, copy=True)),
        ('ensemble', VotingClassifier(
            estimators=[
                ('rf', RandomForestClassifier(n_estimators=100, max_depth=7,
                                            class_weight='balanced', random_state=42)),
                ('xgb', XGBClassifier(n_estimators=100, learning_rate=0.05,
                                     eval_metric='logloss', random_state=42, tree_method='hist'))
            ],
            voting='soft'
        ))
    ])

# 3. Feature Importance Visualization
def plot_feature_importance(pipeline, X_train, feature_names, model_name):
    rf = pipeline.named_steps['ensemble'].estimators_[0]  # RandomForest
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1][:10]

    plt.figure(figsize=(10, 6))
    plt.bar(range(10), importances[indices], align='center')
    plt.xticks(range(10), [feature_names[i] for i in indices], rotation=45)
    plt.title(f'Top 10 Feature Importances ({model_name})')
    plt.tight_layout()
    plt.savefig(f'feature_importance_{model_name.lower()}.png')
    plt.close()
    print(f"Feature importance plot saved to feature_importance_{model_name.lower()}.png")

# 4. Plotting Metrics for Individual Models and Ensemble
def plot_model_metrics(pipeline, X_test, y_test, model_name, estimator=None):
    # If estimator is provided, use it (for individual models); otherwise, use pipeline (ensemble)
    if estimator:
        y_pred = estimator.predict(X_test)
        y_proba = estimator.predict_proba(X_test)
    else:
        y_pred = pipeline.predict(X_test)
        y_proba = pipeline.predict_proba(X_test)

    # Compute metrics
    acc = accuracy_score(y_test, y_pred)
    loss = log_loss(y_test, y_proba)

    # Create plots
    plt.figure(figsize=(15, 5))

    # Confusion Matrix
    plt.subplot(1, 3, 1)
    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(cm, display_labels=['Left Fist', 'Both Feet']).plot(ax=plt.gca())
    plt.title(f'Confusion Matrix ({model_name})')

    # ROC Curve
    plt.subplot(1, 3, 2)
    RocCurveDisplay.from_predictions(y_test, y_proba[:, 1], ax=plt.gca(), name=model_name)
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r')
    plt.title(f'ROC Curve ({model_name})')

    # Precision-Recall Curve
    plt.subplot(1, 3, 3)
    PrecisionRecallDisplay.from_predictions(y_test, y_proba[:, 1], ax=plt.gca(), name=model_name)
    plt.title(f'Precision-Recall Curve ({model_name})')

    plt.tight_layout()
    plt.savefig(f'metrics_{model_name.lower()}.png')
    plt.close()
    print(f"Metrics plot saved to metrics_{model_name.lower()}.png")

    return {'accuracy': acc, 'log_loss': loss}

# 5. Training and Evaluation
def train_and_evaluate(pipeline, X_train, y_train, X_test, y_test):
    # Fit feature pipeline (scaler + PCA)
    feature_pipeline = Pipeline(pipeline.steps[:-1])
    X_train_transformed = feature_pipeline.fit_transform(X_train, y_train)
    X_test_transformed = feature_pipeline.transform(X_test)

    # Fit the full pipeline
    pipeline.fit(X_train, y_train)

    # Evaluate individual models
    rf = pipeline.named_steps['ensemble'].estimators_[0]  # RandomForest
    xgb = pipeline.named_steps['ensemble'].estimators_[1]  # XGBoost

    # Store metrics
    metrics = {}

    # Individual models (predict on transformed data)
    print("\nEvaluating individual models...")
    metrics['RandomForest'] = plot_model_metrics(pipeline, X_test_transformed, y_test, 'RandomForest', rf)
    metrics['XGBoost'] = plot_model_metrics(pipeline, X_test_transformed, y_test, 'XGBoost', xgb)

    # Ensemble model
    print("\nEvaluating ensemble model...")
    metrics['Ensemble'] = plot_model_metrics(pipeline, X_test, y_test, 'Ensemble')

    # Print results
    print("\nModel Performance:")
    for model_name, result in metrics.items():
        print(f"{model_name}:")
        print(f"  Accuracy: {result['accuracy']:.4f}")
        print(f"  Log Loss: {result['log_loss']:.4f}")

    # Save metrics to CSV
    metrics_df = pd.DataFrame({
        'Model': metrics.keys(),
        'Accuracy': [m['accuracy'] for m in metrics.values()],
        'Log_Loss': [m['log_loss'] for m in metrics.values()]
    })
    metrics_df.to_csv('model_metrics.csv', index=False)
    print("\nMetrics saved to model_metrics.csv")

    return metrics

# Random Forest Plot
def plot_rf_loss_accuracy(rf_metrics):
    epochs_rf = range(1, len(rf_metrics['train_loss']) + 1)

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_rf, rf_metrics['train_loss'], label='Train Loss', color='blue')
    plt.plot(epochs_rf, rf_metrics['test_loss'], label='Test Loss', color='orange')
    plt.title('Random Forest Loss')
    plt.xlabel('Trees')
    plt.ylabel('Log Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epochs_rf, rf_metrics['train_acc'], label='Train Accuracy', color='blue')
    plt.plot(epochs_rf, rf_metrics['test_acc'], label='Test Accuracy', color='orange')
    plt.title('Random Forest Accuracy')
    plt.xlabel('Trees')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('rf_loss_accuracy.png')
    plt.close()
    print("Random Forest plot saved as rf_loss_accuracy.png")

# XGBoost Plot
def plot_xgb_loss_accuracy(xgb_metrics):
    epochs_xgb = range(1, len(xgb_metrics['train_loss']) + 1)

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_xgb, xgb_metrics['train_loss'], label='Train Loss', color='blue')
    plt.plot(epochs_xgb, xgb_metrics['test_loss'], label='Test Loss', color='orange')
    plt.title('XGBoost Loss')
    plt.xlabel('Boosting Rounds')
    plt.ylabel('Log Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epochs_xgb, xgb_metrics['train_acc'], label='Train Accuracy', color='blue')
    plt.plot(epochs_xgb, xgb_metrics['test_acc'], label='Test Accuracy', color='orange')
    plt.title('XGBoost Accuracy')
    plt.xlabel('Boosting Rounds')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('xgb_loss_accuracy.png')
    plt.close()
    print("XGBoost plot saved as xgb_loss_accuracy.png")

# Ensemble Plot
def plot_ensemble_loss_accuracy(ensemble_metrics):
    epochs_ensemble = range(1, len(ensemble_metrics['train_loss']) + 1)

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_ensemble, ensemble_metrics['train_loss'], label='Train Loss', color='blue')
    plt.plot(epochs_ensemble, ensemble_metrics['test_loss'], label='Test Loss', color='orange')
    plt.title('Ensemble Loss')
    plt.xlabel('Iterations')
    plt.ylabel('Log Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epochs_ensemble, ensemble_metrics['train_acc'], label='Train Accuracy', color='blue')
    plt.plot(epochs_ensemble, ensemble_metrics['test_acc'], label='Test Accuracy', color='orange')
    plt.title('Ensemble Accuracy')
    plt.xlabel('Iterations')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('ensemble_loss_accuracy.png')
    plt.close()
    print("Ensemble plot saved as ensemble_loss_accuracy.png")


# 7. Main Execution
def main():
    feature_file = 'eeg_features.csv'
    model_file = 'trained_model.joblib'

    # Load features
    try:
        X, y, feature_names = load_features(feature_file)
    except Exception as e:
        print(f"Failed to load features: {str(e)}")
        return

    if X.shape[0] < 50:
        print(f"Warning: Only {X.shape[0]} samples available. Consider more data.")

    # Create pipeline
    pipeline = create_training_pipeline()

    # Cross-validation (ensemble only)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    try:
        print("Starting cross-validation...")
        scores = cross_val_score(
            pipeline, X, y,
            cv=cv, n_jobs=1,
            verbose=1,
            scoring='accuracy',
            error_score='raise'
        )
        print(f"\nCross-Validation Scores: {scores}")
        print(f"Mean CV Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")
    except Exception as e:
        print(f"\nCross-validation failed: {str(e)}")
        return

    # Train and evaluate
    print("\nTraining final model...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    metrics = train_and_evaluate(pipeline, X_train, y_train, X_test, y_test)

    # Save the trained model
    try:
        joblib.dump(pipeline, model_file)
        print(f"Trained model saved to {model_file}")
    except Exception as e:
        print(f"Failed to save model: {str(e)}")

    # Compute metrics for loss and accuracy plots
    feature_pipeline = Pipeline(pipeline.steps[:-1])
    X_train_transformed = feature_pipeline.fit_transform(X_train, y_train)
    X_test_transformed = feature_pipeline.transform(X_test)

    # Random Forest metrics
    rf_metrics = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}
    for n_trees in range(1, 101, 5):  # Step by 5 for efficiency
        rf = RandomForestClassifier(n_estimators=n_trees, max_depth=7,
                                   class_weight='balanced', random_state=42)
        rf.fit(X_train_transformed, y_train)
        rf_metrics['train_loss'].append(log_loss(y_train, rf.predict_proba(X_train_transformed)))
        rf_metrics['test_loss'].append(log_loss(y_test, rf.predict_proba(X_test_transformed)))
        rf_metrics['train_acc'].append(accuracy_score(y_train, rf.predict(X_train_transformed)))
        rf_metrics['test_acc'].append(accuracy_score(y_test, rf.predict(X_test_transformed)))

    # XGBoost metrics
    xgb = XGBClassifier(n_estimators=100, learning_rate=0.05, eval_metric='logloss',
                       random_state=42, tree_method='hist')
    eval_set = [(X_train_transformed, y_train), (X_test_transformed, y_test)]
    xgb.fit(X_train_transformed, y_train, eval_set=eval_set, verbose=False)
    xgb_metrics = {
        'train_loss': xgb.evals_result()['validation_0']['logloss'],
        'test_loss': xgb.evals_result()['validation_1']['logloss'],
        'train_acc': [],
        'test_acc': []
    }
    for i in range(len(xgb_metrics['train_loss'])):
        xgb.set_params(n_estimators=i+1)
        xgb.fit(X_train_transformed, y_train)
        xgb_metrics['train_acc'].append(accuracy_score(y_train, xgb.predict(X_train_transformed)))
        xgb_metrics['test_acc'].append(accuracy_score(y_test, xgb.predict(X_test_transformed)))

    # Ensemble metrics (simulate by training on increasing data subsets)
    ensemble_metrics = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}
    n_iterations = 20
    data_sizes = np.linspace(0.1, 1.0, n_iterations)
    for frac in data_sizes:
        idx = np.random.choice(len(X_train_transformed), size=int(len(X_train_transformed) * frac), replace=False)
        X_subset = X_train_transformed[idx]
        y_subset = y_train[idx]
        pipeline.fit(X_subset, y_subset)
        ensemble_metrics['train_loss'].append(log_loss(y_train, pipeline.predict_proba(X_train_transformed)))
        ensemble_metrics['test_loss'].append(log_loss(y_test, pipeline.predict_proba(X_test_transformed)))
        ensemble_metrics['train_acc'].append(accuracy_score(y_train, pipeline.predict(X_train_transformed)))
        ensemble_metrics['test_acc'].append(accuracy_score(y_test, pipeline.predict(X_test_transformed)))

    # Plot feature importance for Random Forest
    plot_feature_importance(pipeline, X_train, feature_names, 'RandomForest')
    plot_rf_loss_accuracy(rf_metrics)
    plot_xgb_loss_accuracy(xgb_metrics)
    plot_ensemble_loss_accuracy(ensemble_metrics)
    print("Plots saved for Random Forest, XGBoost, and Ensemble models.")
    
if __name__ == '__main__':
    main()

Loaded features shape: (4932, 55)
Class distribution - Left Fist: 2478, Both Feet: 2454
Starting cross-validation...

Cross-Validation Scores: [0.82674772 0.87639311 0.85801217 0.87119675 0.85598377]
Mean CV Accuracy: 0.8577 ± 0.0173

Training final model...

Evaluating individual models...
Metrics plot saved to metrics_randomforest.png
Metrics plot saved to metrics_xgboost.png

Evaluating ensemble model...
Metrics plot saved to metrics_ensemble.png

Model Performance:
RandomForest:
  Accuracy: 0.7903
  Log Loss: 0.5540
XGBoost:
  Accuracy: 0.8369
  Log Loss: 0.3931
Ensemble:
  Accuracy: 0.8470
  Log Loss: 0.4655

Metrics saved to model_metrics.csv
Trained model saved to trained_model.joblib
Feature importance plot saved to feature_importance_randomforest.png
Random Forest plot saved as rf_loss_accuracy.png
XGBoost plot saved as xgb_loss_accuracy.png
Ensemble plot saved as ensemble_loss_accuracy.png
Plots saved for Random Forest, XGBoost, and Ensemble models.
